In [4]:
import pandas as pd
import numpy as np
import gc
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.preprocessing import StandardScaler

tqdm.pandas(desc="my bar!")
pd.set_option("max_columns", 100)
pd.set_option("max_rows", 100)

def disp_full(x, drows=False, dcols=True):
    if drows:
        pd.set_option('display.max_rows', x.shape[0])
    if dcols:
        pd.set_option('display.max_columns', x.shape[1])
    display(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')


def wlogloss(targets, preds):
    target_cols = [
        "any",
        "epidural",
        "subdural",
        "subarachnoid",
        "intraventricular",
        "intraparenchymal",
    ]
    res = 0
    for col in target_cols:
        res += log_loss(targets[col], preds[col + "_pred"])
        if col == "any":
            res += log_loss(targets[col], preds[col + "_pred"])
    res /= 7
    return res


def make_feats(df):
    df["ImagePositionPatient_2"] = df["ImagePositionPatient"].progress_apply(
        lambda x: x[2]
    )
    df = df.merge(
        df.groupby(["StudyInstanceUID"])["ImagePositionPatient_2"]
        .agg(position_min="min", position_max="max")
        .reset_index(),
        on="StudyInstanceUID",
    )
    df["position"] = (df["ImagePositionPatient_2"] - df["position_min"]) / (
        df["position_max"] - df["position_min"]
    )
    res = df.sort_values(by=["StudyInstanceUID", "position"])
    return res


def pred_agg1_train(df):
    new_feats = []
    pred_cols = [column for column in df.columns if "pred" in column]
    print(pred_cols)
    for c in pred_cols:
        tmp = (
            df.groupby(["StudyInstanceUID"])[c]
            .agg(["min", "max", "mean", "std"])
            .reset_index()
        )
        tmp.columns = [
            "StudyInstanceUID",
            c + "_min",
            c + "_max",
            c + "_mean",
            c + "_std",
        ]
        if c != "any_pred_tsukiyama_inceotionv4":
            del tmp["StudyInstanceUID"]
        new_feats.append(tmp)
    new_feats = pd.concat(new_feats, axis=1)
    df = pd.merge(df, new_feats, on="StudyInstanceUID", how="left")
    for c in pred_cols:
        df[c + "_diff"] = df[c] - df[c + "_mean"]
        df[c + "_div"] = df[c] / df[c + "_mean"]
        df[c + "_scaled"] = (df[c] - df[c + "_mean"]) / df[c + "_std"]
    return df


def pred_agg1_test(df):
    new_feats = []
    pred_cols = [column for column in df.columns if "pred" in column]
    print(pred_cols)
    for c in pred_cols:
        tmp = (
            df.groupby(["StudyInstanceUID"])[c]
            .agg(["min", "max", "mean", "std"])
            .reset_index()
        )
        tmp.columns = [
            "StudyInstanceUID",
            c + "_min",
            c + "_max",
            c + "_mean",
            c + "_std",
        ]
        if "any_pred" not in c:
            del tmp["StudyInstanceUID"]
        new_feats.append(tmp)
    new_feats = pd.concat(new_feats, axis=1)
    df = pd.merge(df, new_feats, on="StudyInstanceUID", how="left")
    for c in pred_cols:
        df[c + "_diff"] = df[c] - df[c + "_mean"]
        df[c + "_div"] = df[c] / df[c + "_mean"]
        df[c + "_scaled"] = (df[c] - df[c + "_mean"]) / df[c + "_std"]
    return df


def pred_agg2(df):
    pred_cols = [column for column in df.columns if "pred" in column]
    print(pred_cols)
    a1 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(3, min_periods=1, center=True)
        .mean()
        .values
    )
    a2 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(5, min_periods=1, center=True)
        .mean()
        .values
    )
    a3 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(1, min_periods=1, center=True)
        .mean()
        .values
    )
    new_feats1 = pd.DataFrame(a1, columns=[c + "_3roll" for c in pred_cols])
    new_feats2 = pd.DataFrame(a2, columns=[c + "_5roll" for c in pred_cols])
    new_feats3 = pd.DataFrame(a1 - a3, columns=[c + "_3rolldiff" for c in pred_cols])
    new_feats4 = pd.DataFrame(a2 - a3, columns=[c + "_5rolldiff" for c in pred_cols])
    new_feats5 = pd.DataFrame(a1 / a3, columns=[c + "_3rolldiv" for c in pred_cols])
    new_feats6 = pd.DataFrame(a2 / a3, columns=[c + "_5rolldiv" for c in pred_cols])
    new_feats1.index = df.index
    new_feats2.index = df.index
    new_feats3.index = df.index
    new_feats4.index = df.index
    new_feats5.index = df.index
    new_feats6.index = df.index
    df = pd.concat(
        [df, new_feats1, new_feats2, new_feats3, new_feats4, new_feats5, new_feats6],
        axis=1,
    )
    return df

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
#stage = 1
stage = 2

valid_pkl = pd.read_pickle("stage{}_data/train_folds{}.pkl".format(stage, stage))

valid_folds_list = []
for i in range(5):
    valid_folds_list.append(valid_pkl[valid_pkl.fold==i])


In [6]:
target_cols = [
    "any",
    "epidural",
    "subdural",
    "subarachnoid",
    "intraventricular",
    "intraparenchymal",
]

label_to_num = {
    'any': 0,
    'epidural': 1,
    'subdural': 2,
    'subarachnoid': 3,
    'intraventricular': 4,
    'intraparenchymal': 5,
}

shimakoshi_label_dict = {
    "pred_score0": "any_pred",
    "pred_score1": "epidural_pred",
    "pred_score2": "subdural_pred",
    "pred_score3": "subarachnoid_pred",
    "pred_score4": "intraventricular_pred",
    "pred_score5": "intraparenchymal_pred"
}


models = [
    "sasaki_se_resnext101_mixup_410_with_imagenet_norm",
    "sasaki_se_resnext_410",
    "sasaki_senet154_customlabels",
    "shimakoshi_densenet_adj_prediction",
    "shimakoshi_seresnext50_label_smoothing",
    "shimakoshi_seresnext50_label_smoothing_not_any",
    "tsukiyama_inception_resnet_v2",
    "tsukiyama_se_resnext",
    "tsukiyama_xception"
]

target_pred_cols = [col+"_pred" for col in target_cols]

n_tta = 5
n_folds = 5


In [7]:
# valid

for model in models:
    print(model)
    valid_outputs_list = []
    for fold in range(n_folds):
        valid_pkl_list = pd.read_pickle("stage{}_outputs/{}/fold{}_ep2_valid_tta5.pkl".format(stage, model, fold))
        if "shimakoshi" not in model:
            for tta in range(n_tta):
                ids = valid_pkl_list[tta]["ids"]
                outputs = valid_pkl_list[tta]["outputs"]
                _tmp = pd.DataFrame(outputs, columns=target_pred_cols)
                _tmp["ID"] = ids
                if tta == 0:
                    tmp = _tmp
                else:
                    tmp[target_pred_cols] = tmp[target_pred_cols].values+_tmp.set_index("ID").loc[tmp.ID, target_pred_cols].values
                tmp[target_pred_cols] = tmp[target_pred_cols]/n_tta
        else:
            tmp = valid_pkl_list[["ID",]+list(shimakoshi_label_dict.keys())].rename(columns=shimakoshi_label_dict)
        valid_outputs_list.append(tmp)

    valid_dfs = []
    for valid_fold in valid_folds_list:
        target = np.zeros((len(valid_fold), 6))    
        for i, row in tqdm(valid_fold.reset_index(drop=True).iterrows(), total=len(valid_fold)):
            for label in row.labels.split():
                cls = label_to_num[label]
                target[i, cls] = 1.0
        tmp = pd.DataFrame(target, columns=target_cols)
        tmp["ID"] = valid_fold["ID"].values
        valid_dfs.append(tmp)
        
    # predとtrueのmerge
    for i in range(n_folds):
        valid_dfs[i] = pd.merge(valid_dfs[i], valid_outputs_list[i], on="ID", how="left")
        
    # correration check
    for i in range(n_folds):
        disp_full(valid_dfs[i].corr())
    
    for i in range(n_folds):
        print(valid_dfs[i].shape)
        
    # nan check
    cols = target_cols + target_pred_cols
    for fold in range(n_folds):
        print(np.where(np.isnan(valid_dfs[fold][cols].values)))


sasaki_se_resnext101_mixup_410_with_imagenet_norm


100%|██████████| 148048/148048 [00:16<00:00, 9238.46it/s]


,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.158757,0.636149,0.548918,0.467373,0.553552,0.862903,0.148824,0.577864,0.560398,0.473645,0.557729
epidural,0.158757,1.000000,0.047120,0.039915,0.016896,0.067407,0.130567,0.305767,0.147398,0.053625,0.019073,0.081121
subdural,0.636149,0.047120,1.000000,0.200758,0.082653,0.145339,0.530130,0.147744,0.750277,0.249490,0.086943,0.157236
subarachnoid,0.548918,0.039915,0.200758,1.000000,0.230649,0.221846,0.487471,0.047022,0.224065,0.752180,0.249663,0.263631
intraventricular,0.467373,0.016896,0.082653,0.230649,1.000000,0.302432,0.476058,0.021667,0.090676,0.272894,0.879048,0.368470
intraparenchymal,0.553552,0.067407,0.145339,0.221846,0.302432,1.000000,0.547625,0.086701,0.172482,0.308942,0.344268,0.844164
any_pred,0.862903,0.130567,0.530130,0.487471,0.476058,0.547625,1.000000,0.190759,0.684312,0.635394,0.535802,0.634615
epidural_pred,0.148824,0.305767,0.147744,0.047022,0.021667,0.086701,0.190759,1.000000,0.161030,0.066004,0.023434,0.098785
subdural_pred,0.577864,0.147398,0.750277,0.224065,0.090676,0.172482,0.684312,0.161030,1.000000,0.290104,0.105298,0.183570
subarachnoid_pred,0.560398,0.053625,0.249490,0.752180,0.272894,0.308942,0.635394,0.066004,0.290104,1.000000,0.315963,0.349317


,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.157924,0.629319,0.543088,0.461801,0.547305,0.857417,0.235633,0.588936,0.548225,0.453772,0.544506
epidural,0.157924,1.000000,0.033939,0.037101,0.008891,0.022232,0.116512,0.408742,0.132044,0.038146,0.013382,0.044075
subdural,0.629319,0.033939,1.000000,0.171747,0.069875,0.125408,0.506893,0.208205,0.755384,0.209675,0.065945,0.145603
subarachnoid,0.543088,0.037101,0.171747,1.000000,0.201487,0.203442,0.484180,0.090528,0.227087,0.740453,0.213709,0.242287
intraventricular,0.461801,0.008891,0.069875,0.201487,1.000000,0.326625,0.485956,0.049813,0.073334,0.300138,0.868127,0.356234
intraparenchymal,0.547305,0.022232,0.125408,0.203442,0.326625,1.000000,0.561085,0.120053,0.177153,0.292663,0.354756,0.853335
any_pred,0.857417,0.116512,0.506893,0.484180,0.485956,0.561085,1.000000,0.263386,0.673111,0.650344,0.537975,0.647785
epidural_pred,0.235633,0.408742,0.208205,0.090528,0.049813,0.120053,0.263386,1.000000,0.271355,0.105125,0.059052,0.138491
subdural_pred,0.588936,0.132044,0.755384,0.227087,0.073334,0.177153,0.673111,0.271355,1.000000,0.296883,0.080372,0.203245
subarachnoid_pred,0.548225,0.038146,0.209675,0.740453,0.300138,0.292663,0.650344,0.105125,0.296883,1.000000,0.335231,0.329146


,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.157780,0.629680,0.543666,0.462884,0.546621,0.860940,0.217813,0.570967,0.547855,0.456849,0.527149
epidural,0.157780,1.000000,0.053154,0.047818,0.003199,0.031513,0.125408,0.365790,0.143083,0.056011,0.000961,0.040078
subdural,0.629680,0.053154,1.000000,0.187616,0.053738,0.117830,0.517446,0.187642,0.756448,0.224840,0.045445,0.125780
subarachnoid,0.543666,0.047818,0.187616,1.000000,0.217483,0.220874,0.480417,0.089151,0.214531,0.750350,0.221413,0.250031
intraventricular,0.462884,0.003199,0.053738,0.217483,1.000000,0.288464,0.480371,0.040763,0.052324,0.267703,0.866008,0.333129
intraparenchymal,0.546621,0.031513,0.117830,0.220874,0.288464,1.000000,0.545209,0.145263,0.150234,0.304509,0.324135,0.849783
any_pred,0.860940,0.125408,0.517446,0.480417,0.480371,0.545209,1.000000,0.254508,0.668175,0.631764,0.538784,0.604914
epidural_pred,0.217813,0.365790,0.187642,0.089151,0.040763,0.145263,0.254508,1.000000,0.272982,0.121517,0.037767,0.161360
subdural_pred,0.570967,0.143083,0.756448,0.214531,0.052324,0.150234,0.668175,0.272982,1.000000,0.277746,0.049638,0.156156
subarachnoid_pred,0.547855,0.056011,0.224840,0.750350,0.267703,0.304509,0.631764,0.121517,0.277746,1.000000,0.285619,0.331302


...

,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.157924,0.629319,0.543088,0.461801,0.547305,0.845672,0.247212,0.573622,0.568198,0.467147,0.569624
epidural,0.157924,1.000000,0.033939,0.037101,0.008891,0.022232,0.122381,0.415995,0.123923,0.046082,0.012868,0.053102
subdural,0.629319,0.033939,1.000000,0.171747,0.069875,0.125408,0.499428,0.222460,0.734867,0.241971,0.077919,0.171219
subarachnoid,0.543088,0.037101,0.171747,1.000000,0.201487,0.203442,0.478493,0.081087,0.229902,0.719612,0.226638,0.274577
intraventricular,0.461801,0.008891,0.069875,0.201487,1.000000,0.326625,0.480741,0.039979,0.082196,0.315115,0.844816,0.373138
intraparenchymal,0.547305,0.022232,0.125408,0.203442,0.326625,1.000000,0.556725,0.117566,0.165614,0.318631,0.372856,0.832688
any_pred,0.845672,0.122381,0.499428,0.478493,0.480741,0.556725,1.000000,0.286911,0.668038,0.677947,0.556866,0.682344
epidural_pred,0.247212,0.415995,0.222460,0.081087,0.039979,0.117566,0.286911,1.000000,0.281479,0.110411,0.043062,0.155791
subdural_pred,0.573622,0.123923,0.734867,0.229902,0.082196,0.165614,0.668038,0.281479,1.000000,0.343733,0.103668,0.220389
subarachnoid_pred,0.568198,0.046082,0.241971,0.719612,0.315115,0.318631,0.677947,0.110411,0.343733,1.000000,0.373894,0.397382


,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.157780,0.629680,0.543666,0.462884,0.546621,0.850913,0.253847,0.567408,0.566123,0.467287,0.552461
epidural,0.157780,1.000000,0.053154,0.047818,0.003199,0.031513,0.126029,0.379441,0.140587,0.062636,0.002790,0.045858
subdural,0.629680,0.053154,1.000000,0.187616,0.053738,0.117830,0.511843,0.235148,0.737653,0.252090,0.057841,0.140711
subarachnoid,0.543666,0.047818,0.187616,1.000000,0.217483,0.220874,0.476063,0.115586,0.225538,0.725605,0.240951,0.278637
intraventricular,0.462884,0.003199,0.053738,0.217483,1.000000,0.288464,0.464469,0.046955,0.058662,0.259883,0.846502,0.359225
intraparenchymal,0.546621,0.031513,0.117830,0.220874,0.288464,1.000000,0.539748,0.158887,0.148896,0.336263,0.350413,0.835382
any_pred,0.850913,0.126029,0.511843,0.476063,0.464469,0.539748,1.000000,0.298890,0.675688,0.657650,0.541652,0.635043
epidural_pred,0.253847,0.379441,0.235148,0.115586,0.046955,0.158887,0.298890,1.000000,0.326164,0.162611,0.047796,0.183529
subdural_pred,0.567408,0.140587,0.737653,0.225538,0.058662,0.148896,0.675688,0.326164,1.000000,0.324760,0.070486,0.173227
subarachnoid_pred,0.566123,0.062636,0.252090,0.725605,0.259883,0.336263,0.657650,0.162611,0.324760,1.000000,0.302140,0.400774


,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.158138,0.630970,0.543438,0.463081,0.546393,0.851540,0.268608,0.593906,0.556486,0.452230,0.558569
epidural,0.158138,1.000000,0.034031,0.041178,0.024096,0.065879,0.134597,0.322387,0.139672,0.062219,0.025746,0.068647
subdural,0.630970,0.034031,1.000000,0.167936,0.061677,0.130204,0.520317,0.261369,0.746930,0.215579,0.062007,0.149803
subarachnoid,0.543438,0.041178,0.167936,1.000000,0.215992,0.212690,0.467846,0.093304,0.243326,0.716959,0.232596,0.265086
intraventricular,0.463081,0.024096,0.061677,0.215992,1.000000,0.321954,0.467206,0.025150,0.078796,0.289731,0.852905,0.399768
intraparenchymal,0.546393,0.065879,0.130204,0.212690,0.321954,1.000000,0.544033,0.133737,0.183165,0.326150,0.334432,0.838409
any_pred,0.851540,0.134597,0.520317,0.467846,0.467206,0.544033,1.000000,0.332155,0.701190,0.650182,0.522450,0.647361
epidural_pred,0.268608,0.322387,0.261369,0.093304,0.025150,0.133737,0.332155,1.000000,0.345394,0.138508,0.025339,0.144570
subdural_pred,0.593906,0.139672,0.746930,0.243326,0.078796,0.183165,0.701190,0.345394,1.000000,0.312297,0.083498,0.209115
subarachnoid_pred,0.556486,0.062219,0.215579,0.716959,0.289731,0.326150,0.650182,0.138508,0.312297,1.000000,0.325852,0.387507


,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any,1.000000,0.158233,0.631749,0.544930,0.464066,0.547894,0.847368,0.215402,0.559946,0.553206,0.458450,0.557829
epidural,0.158233,1.000000,0.054838,0.036716,0.012213,0.048933,0.131406,0.338029,0.139507,0.063432,0.011551,0.063749
subdural,0.631749,0.054838,1.000000,0.184595,0.042998,0.123437,0.517882,0.229387,0.736238,0.254572,0.043693,0.150526
subarachnoid,0.544930,0.036716,0.184595,1.000000,0.211949,0.227531,0.472869,0.064369,0.199957,0.709704,0.227972,0.287570
intraventricular,0.464066,0.012213,0.042998,0.211949,1.000000,0.325645,0.473314,0.009144,0.052817,0.249078,0.853548,0.398142
intraparenchymal,0.547894,0.048933,0.123437,0.227531,0.325645,1.000000,0.548966,0.070642,0.142223,0.306388,0.363911,0.834870
any_pred,0.847368,0.131406,0.517882,0.472869,0.473314,0.548966,1.000000,0.261714,0.672819,0.649507,0.534662,0.656024
epidural_pred,0.215402,0.338029,0.229387,0.064369,0.009144,0.070642,0.261714,1.000000,0.270132,0.092532,0.010666,0.081825
subdural_pred,0.559946,0.139507,0.736238,0.199957,0.052817,0.142223,0.672819,0.270132,1.000000,0.291233,0.053571,0.179333
subarachnoid_pred,0.553206,0.063432,0.254572,0.709704,0.249078,0.306388,0.649507,0.092532,0.291233,1.000000,0.272121,0.378435


(148198, 13)
(149467, 13)
(147867, 13)
(150197, 13)
(148048, 13)
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [8]:
# test

for model in models:
    print(model)
    test_outputs_list = []
    for fold in range(n_folds):
        test_pkl_list = pd.read_pickle("stage{}_outputs/{}/fold{}_ep2_test_tta5.pkl".format(stage, model, fold))
        if "shimakoshi" not in model:
            for tta in range(n_tta):
                ids = test_pkl_list[tta]["ids"]
                outputs = test_pkl_list[tta]["outputs"]
                _tmp = pd.DataFrame(outputs, columns=target_pred_cols)
                _tmp["ID"] = ids
                if tta == 0:
                    tmp = _tmp
                else:
                    tmp[target_pred_cols] = tmp[target_pred_cols].values+_tmp.set_index("ID").loc[tmp.ID, target_pred_cols].values
                tmp[target_pred_cols] = tmp[target_pred_cols]/n_tta
        else:
            tmp = test_pkl_list[["ID",]+list(shimakoshi_label_dict.keys())].rename(columns=shimakoshi_label_dict)
        test_outputs_list.append(tmp)

    # correration check
    for i in range(n_folds):
        disp_full(test_outputs_list[i].corr())
    
    for i in range(n_folds):
        if i == 0:
            tmp = test_outputs_list[i].any_pred.values.reshape(-1, 1)
        else:
            tmp = np.concatenate((tmp, test_outputs_list[i].any_pred.values.reshape(-1, 1)), axis=1)
    tmp = pd.DataFrame(tmp, columns=np.arange(n_folds))
    disp_full(tmp.corr())
        
    # nan check
    cols = target_pred_cols
    for fold in range(n_folds):
        print(np.where(np.isnan(test_outputs_list[fold][cols].values)))


sasaki_se_resnext101_mixup_410_with_imagenet_norm


,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.182819,0.673049,0.619768,0.559624,0.623795
epidural_pred,0.182819,1.000000,0.158815,0.068817,0.026475,0.099834
subdural_pred,0.673049,0.158815,1.000000,0.289554,0.080675,0.165396
subarachnoid_pred,0.619768,0.068817,0.289554,1.000000,0.305150,0.334026
intraventricular_pred,0.559624,0.026475,0.080675,0.305150,1.000000,0.426217
intraparenchymal_pred,0.623795,0.099834,0.165396,0.334026,0.426217,1.000000


,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.258035,0.668573,0.641301,0.561333,0.642109
epidural_pred,0.258035,1.000000,0.270030,0.109582,0.049859,0.173829
subdural_pred,0.668573,0.270030,1.000000,0.311955,0.064666,0.203790
subarachnoid_pred,0.641301,0.109582,0.311955,1.000000,0.328288,0.333301
intraventricular_pred,0.561333,0.049859,0.064666,0.328288,1.000000,0.396256
intraparenchymal_pred,0.642109,0.173829,0.203790,0.333301,0.396256,1.000000


,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.244244,0.661974,0.622802,0.565270,0.608874
epidural_pred,0.244244,1.000000,0.251819,0.110453,0.025480,0.159592
subdural_pred,0.661974,0.251819,1.000000,0.302687,0.046754,0.159844
subarachnoid_pred,0.622802,0.110453,0.302687,1.000000,0.270172,0.324023
intraventricular_pred,0.565270,0.025480,0.046754,0.270172,1.000000,0.400416
intraparenchymal_pred,0.608874,0.159592,0.159844,0.324023,0.400416,1.000000


,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.274542,0.667934,0.619133,0.558609,0.649203
epidural_pred,0.274542,1.000000,0.280423,0.106258,0.009028,0.109948
subdural_pred,0.667934,0.280423,1.000000,0.307334,0.057140,0.176145
subarachnoid_pred,0.619133,0.106258,0.307334,1.000000,0.315632,0.348744
intraventricular_pred,0.558609,0.009028,0.057140,0.315632,1.000000,0.420017
intraparenchymal_pred,0.649203,0.109948,0.176145,0.348744,0.420017,1.000000


...

,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.275989,0.666264,0.646170,0.570799,0.638250
epidural_pred,0.275989,1.000000,0.284993,0.138811,0.039269,0.165115
subdural_pred,0.666264,0.284993,1.000000,0.343040,0.075392,0.183335
subarachnoid_pred,0.646170,0.138811,0.343040,1.000000,0.285016,0.386478
intraventricular_pred,0.570799,0.039269,0.075392,0.285016,1.000000,0.447244
intraparenchymal_pred,0.638250,0.165115,0.183335,0.386478,0.447244,1.000000


,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.302357,0.682365,0.640947,0.553635,0.650314
epidural_pred,0.302357,1.000000,0.322628,0.123845,0.019102,0.132647
subdural_pred,0.682365,0.322628,1.000000,0.305780,0.073015,0.202045
subarachnoid_pred,0.640947,0.123845,0.305780,1.000000,0.317614,0.366262
intraventricular_pred,0.553635,0.019102,0.073015,0.317614,1.000000,0.438903
intraparenchymal_pred,0.650314,0.132647,0.202045,0.366262,0.438903,1.000000


,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred
any_pred,1.000000,0.246116,0.659740,0.649476,0.561337,0.658531
epidural_pred,0.246116,1.000000,0.247266,0.108154,0.018709,0.105124
subdural_pred,0.659740,0.247266,1.000000,0.308739,0.059136,0.184370
subarachnoid_pred,0.649476,0.108154,0.308739,1.000000,0.264672,0.377141
intraventricular_pred,0.561337,0.018709,0.059136,0.264672,1.000000,0.459612
intraparenchymal_pred,0.658531,0.105124,0.184370,0.377141,0.459612,1.000000


,0,1,2,3,4
0,1.000000,0.977892,0.979948,0.979148,0.976972
1,0.977892,1.000000,0.980672,0.981217,0.981149
2,0.979948,0.980672,1.000000,0.981993,0.979128
3,0.979148,0.981217,0.981993,1.000000,0.980551
4,0.976972,0.981149,0.979128,0.980551,1.000000


(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [66]:
model = "shimakoshi_densenet_adj_prediction"

print(model)
test_outputs_list = []
for fold in range(n_folds):
    test_pkl_list = pd.read_pickle("stage{}_outputs/{}/fold{}_ep2_test_tta5.pkl".format(stage, model, fold))
    tmp = test_pkl_list[["ID",]+list(shimakoshi_label_dict.keys())]
    test_outputs_list.append(tmp)

# nan check
cols = list(shimakoshi_label_dict.keys())
for fold in range(n_folds):
    print(np.where(np.isnan(test_outputs_list[fold][cols].values)))
    
# 0でnan埋め 
cols = list(shimakoshi_label_dict.keys())
for fold in range(n_folds):
    print(np.sum(np.isnan(test_outputs_list[fold][cols].values)))
    test_outputs_list[fold].loc[:, cols] = test_outputs_list[fold].loc[:, cols].fillna(0)
    print(np.sum(np.isnan(test_outputs_list[fold][cols].values)))

shimakoshi_densenet_adj_prediction
(array([   125,    125,    125, ..., 121199, 121199, 121199]), array([0, 1, 2, ..., 3, 4, 5]))
(array([   125,    125,    125, ..., 121199, 121199, 121199]), array([0, 1, 2, ..., 3, 4, 5]))
(array([   125,    125,    125, ..., 121199, 121199, 121199]), array([0, 1, 2, ..., 3, 4, 5]))
(array([   125,    125,    125, ..., 121199, 121199, 121199]), array([0, 1, 2, ..., 3, 4, 5]))
(array([   125,    125,    125, ..., 121199, 121199, 121199]), array([0, 1, 2, ..., 3, 4, 5]))
15180
0
15180
0
15180
0
15180
0
15180
0


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [73]:
if "shimakoshi" in model:
    for fold in range(n_folds):
        print("stage{}_outputs/{}/fold{}_ep2_test_tta5.pkl".format(stage, model, fold))
        test_outputs_list[fold].to_pickle("stage{}_outputs/{}/fold{}_ep2_test_tta5.pkl".format(stage, model, fold))

stage2_outputs/shimakoshi_densenet_adj_prediction/fold0_ep2_test_tta5.pkl
stage2_outputs/shimakoshi_densenet_adj_prediction/fold1_ep2_test_tta5.pkl
stage2_outputs/shimakoshi_densenet_adj_prediction/fold2_ep2_test_tta5.pkl
stage2_outputs/shimakoshi_densenet_adj_prediction/fold3_ep2_test_tta5.pkl
stage2_outputs/shimakoshi_densenet_adj_prediction/fold4_ep2_test_tta5.pkl
